In [1]:
cd "Model/Segmentacion"


/home/jupyter-pcarmona@alges.cl/Model/Segmentacion


In [8]:
ls

data.ipynb         unetmIoU-0.211.pt  unetmIoU-0.285.pt  unetmIoU-0.293.pt
data.py            unetmIoU-0.214.pt  unetmIoU-0.286.pt  unetmIoU-0.296.pt
optuna.db          unetmIoU-0.215.pt  unetmIoU-0.287.pt  unetmIoU-0.734.pt
__pycache__/       unetmIoU-0.218.pt  unetmIoU-0.288.pt  unetmIoU-0.735.pt
txt.txt            unetmIoU-0.277.pt  unetmIoU-0.289.pt  unetmIoU-0.737.pt
unetmIoU-0.204.pt  unetmIoU-0.283.pt  unetmIoU-0.290.pt  unetmIoU-0.739.pt
unetmIoU-0.209.pt  unetmIoU-0.284.pt  unetmIoU-0.291.pt  unetmIoU-0.741.pt


In [9]:
from data import *
from torch.utils.data import DataLoader
from torchvision import transforms


#datasets
BATCH_SIZE = 4
mode="baches"
IMAGE_PATH, MASK_PATH = getpaths(mode)
df=create_df(mode)
train_set = OperationDataset(IMAGE_PATH, MASK_PATH, df['id'].values)
val_set = OperationDataset(IMAGE_PATH, MASK_PATH, df['id'].values)

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=True)

In [5]:
import optuna
import segmentation_models_pytorch as smp
def Objective(trial):
    ENCODER_NAME = 'resnet18'#'timm-regnetx_002'#trial.suggest_categorical('encoder',['resnet50','resnet18','timm-efficientnet-b1'])#'mobilenet_v2'  # 'mobilenet_v2', 'resnet50', 'resnet34'
    ENCODER_WEIGHTS = 'imagenet'  # None, 'imagenet', 'ssl', 'swsl'
    EPOCHS = 10
    ENCODER_DEPTH = 3
    POOLING = 'avg'  #  'avg', 'max'
    DROPOUT = 0.1
    CHANS=3

    model=getModel('unet++')#trial.suggest_categorical('model',['unet++','fpn','linknet']))
    LR = 0.002#trial.suggest_float('lr', 1e-5, 1e-2)
    WEIGHT_DECAY = 0.004#trial.suggest_float('weight_decay', 1e-5, 1e-2)
    #criterion = trial.suggest_categorical('criterion', ['LV','DL','Focal','Tversky'])
    criterion = getCriterion('Focal')#trial.suggest_categorical('criterion', ['LV','DL','Focal','Tversky']))
    
    # optimizer = torch.optim.Adam([dict(params=model.parameters(), lr=LR),])
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, LR, epochs=EPOCHS,
                                                steps_per_epoch=len(train_loader))

    history = fit(EPOCHS, model, train_loader, val_loader, criterion, optimizer, sched)
    return history['val_miou'][-1]
nt=60
study_name = 'optuna4'  # Unique identifier of the study.
storage_name = 'sqlite:///optuna.db'  # Storage in DB.
study = optuna.create_study(direction='maximize',load_if_exists=True, study_name=study_name, storage=storage_name, sampler=optuna.samplers.TPESampler(seed=42), pruner=optuna.pruners.MedianPruner(n_startup_trials=int(nt/3), n_warmup_steps=int(nt/3)))
study.optimize(Objective, n_trials=nt)


[I 2023-04-21 10:56:08,935] Using an existing study with name 'optuna4' instead of creating a new one.


  0%|          | 0/82 [00:00<?, ?it/s]

[W 2023-04-21 10:56:14,461] Trial 32 failed with parameters: {} because of the following error: RuntimeError('CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 7.93 GiB total capacity; 6.58 GiB already allocated; 4.06 MiB free; 7.25 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF').
Traceback (most recent call last):
  File "/home/jupyter-pcarmona@alges.cl/.local/lib/python3.9/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_65606/2675833127.py", line 24, in Objective
    history = fit(EPOCHS, model, train_loader, val_loader, criterion, optimizer, sched)
  File "/home/jupyter-pcarmona@alges.cl/Model/Segmentacion/data.py", line 233, in fit
    loss = criterion(output, mask)
  File "/home/jupyter-pcarmona@alges.cl/.local/lib/python3.9/site-packages/torch/nn/modu

RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 7.93 GiB total capacity; 6.58 GiB already allocated; 4.06 MiB free; 7.25 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [40]:
import matplotlib.pyplot as plt
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
x=OperationDataset(getpaths('baches')[0],getpaths('baches')[1],df['id'].values)
model="unetmIoU-0.741.pt"
model=torch.load(model)
model.to(device)
model.eval()
len(x)

326

In [46]:
from ipywidgets import interact
import torch
def load(id):
    img=np.load(getpaths(mode)[0]+str(id)+'.npz')['arr_0'][:,:,:3]
    mask=np.load(getpaths(mode)[1]+str(id)+'.npz')['arr_0']
    return img,mask
def plot(id,pred=None):
    img,mask=load(id)
    if pred is not None:
        fig, ax =  plt.subplots(1, 3, figsize=(18,18))
    else:
        fig, ax =  plt.subplots(1, 2, figsize=(18,18))
    #set fixed colors of mask and pred for 7 classes
    colors = np.array([[0, 0, 0], [0, 0, 255], [0, 255, 0], [255, 0, 0], [255, 255, 0], [255, 0, 255], [0, 255, 255]])
    mask = colors[mask]
    if pred is not None:
        pred = colors[pred]
    ax[0].imshow(img)
    ax[1].imshow(mask)
    if pred is not None:
        ax[2].imshow(pred)
    plt.show()
def pred(id):
    img,mask=load(id)
    img=torch.from_numpy(img).permute(2,0,1).unsqueeze(0).float()
    pred=model(img.to(device))
    pred=torch.argmax(pred, dim=1).cpu().squeeze().detach().numpy()
    plot(id,pred)
interact(pred, id=(0,len(x)-1))

interactive(children=(IntSlider(value=162, description='id', max=325), Output()), _dom_classes=('widget-intera…

<function __main__.pred(id)>